## Module 2 Project 

link to spreadsheet: https://docs.google.com/spreadsheets/d/1kL1rU6_G7Klr78RxtadjEI4dSzmYr3hZEPXT-HaghnA/edit#gid=453759461

link to github: https://github.com/petehumphries/Flatiron_School-Module_2_Proj.git


### Executive Summary

This notebook is split into the following sections:

    1) Connect to the SQL database
    2) Extract + Transform + Load
    3) 


Upon completion of this lab, each unique team in this dataset should have a record in the MongoDB instance containing the following information:

The name of the team
The total number of goals scored by the team during the 2011 season
The total number of wins the team earned during the 2011 season
A histogram visualization of the team's wins and losses for the 2011 season (store the visualization directly)

The team's win percentage on days where it was raining during games in the 2011 season.

### Inputs

    All team names
    All Match results for 2011
    All goals scored for 2011
    All match dates 
    
    Weather

SQL DB: Name of team, goals scored during the 2011 season, total number of wins
Weather: From the Weather API - Berlin only
Join by date

List the match days => max = 365, likely below 200

### Tables:

There are the following Tables: 

Matches, Teams, Unique Teams, Teams_in_Matches, FlatView, FlatView_Advanced, FlatView_Chrono_TeamOrder_Reduced,  

Match_ID (int): unique ID per match
Div (str): identifies the division the match was played in (D1 = Bundesliga, D2 = Bundesliga 2, E0 = English Premier League)
Season (int): Season the match took place in (usually covering the period of August till May of the following year)
Date (str): Date of the match
HomeTeam (str): Name of the home team
AwayTeam (str): Name of the away team
FTHG (int) (Full Time Home Goals): Number of goals scored by the home team
FTAG (int) (Full Time Away Goals): Number of goals scored by the away team
FTR (str) (Full Time Result): 3-way result of the match (H = Home Win, D = Draw, A = Away Win)

#### Table: Teams

Season (str): Football season for which the data is valid
TeamName (str): Name of the team the data concerns
KaderHome (str): Number of Players in the squad
AvgAgeHome (str): Average age of players
ForeignPlayersHome (str): Number of foreign players (non-German, non-English respectively) playing for the team
OverallMarketValueHome (str): Overall market value of the team pre-season in EUR (based on data from transfermarkt.de)
AvgMarketValueHome (str): Average market value (per player) of the team pre-season in EUR (based on data from transfermarkt.de)
StadiumCapacity (str): Maximum stadium capacity of the team's home stadium

#### Table: Unique Teams

TeamName (str): Name of a team
Unique_Team_ID (int): Unique identifier for each team

#### Table: Teams_in_Matches

Match_ID (int): Unique match ID
Unique_Team_ID (int): Unique team ID (This table is used to easily retrieve each match a given team has played in)
Based on these tables I created a couple of views which I used as input for my machine learning models:

#### View: FlatView

Combination of all matches with the respective additional data from Teams table for both home and away team.

#### View: FlatView_Advanced

Same as Flatview but also includes Unique_Team_ID and Unique_Team in order to easily retrieve all matches played by a team in chronological order.

#### View: FlatView_Chrono_TeamOrder_Reduced

Similar to Flatview_Advanced, however missing the additional attributes from team in order to have a longer history including years 1993 - 2004. Especially interesting if one is only interested in analyzing winning/loosing streaks.
Match_ID (int): unique ID per match
Div (str): identifies the division the match was played in (D1 = Bundesliga, D2 = Bundesliga 2, E0 = English Premier League)
Season (int): Season the match took place in (usually covering the period of August till May of the following year)
Date (str): Date of the match
HomeTeam (str): Name of the home team
AwayTeam (str): Name of the away team
FTHG (int) (Full Time Home Goals): Number of goals scored by the home team
FTAG (int) (Full Time Away Goals): Number of goals scored by the away team
FTR (str) (Full Time Result): 3-way result of the match (H = Home Win, D = Draw, A = Away Win)

In [3]:
import sqlite3
conn = sqlite3.connect('database.sqlite')
cur = conn.cursor()

Extracting all games from the Matches table, filtering on the 2011 season

In [18]:
cur.execute("""SELECT * FROM MATCHES WHERE Season == '2011' LIMIT 5000;""")
len(cur.fetchall())  #length filtered on 2011 season

992

Excluding the Enplish Premiership Games from the import using AND Div <> 'E0' condition. 612 entries

In [19]:
cur.execute("""SELECT * FROM MATCHES WHERE Season == '2011' AND Div <> 'E0';""")

In [20]:
len(cur.fetchall()) # 612 rows expected

612

In [21]:
import pandas as pd

To exclude the English Premier League Matches where Div = E0

In [22]:
cur.execute("""SELECT * FROM MATCHES WHERE Season == '2011' AND Div <> 'E0';""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head(1000)

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D
...,...,...,...,...,...,...,...,...,...
607,1699,D2,2011,2011-07-24,Paderborn,Fortuna Dusseldorf,1,1,D
608,1700,D2,2011,2011-12-10,Bochum,Fortuna Dusseldorf,1,1,D
609,1701,D2,2011,2011-08-15,Ein Frankfurt,Fortuna Dusseldorf,1,1,D
610,1702,D2,2011,2011-08-27,Aachen,Fortuna Dusseldorf,0,0,D


Creating a new column to show "Match_Winner" 

In [55]:
import numpy as np

col         = 'FTR'
conditions  = [ df[col] == 'H', df[col] == 'A', df[col] == 'D' ]
choices     = [ df['HomeTeam'], df['AwayTeam'], 'Draw' ]

df["Match_Winner"] = np.select(conditions, choices, default=np.nan) 

In [56]:
df

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Match_Winner
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,Bayern Munich
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,Bayern Munich
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,Bayern Munich
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H,Mainz
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D,Draw
...,...,...,...,...,...,...,...,...,...,...
607,1699,D2,2011,2011-07-24,Paderborn,Fortuna Dusseldorf,1,1,D,Draw
608,1700,D2,2011,2011-12-10,Bochum,Fortuna Dusseldorf,1,1,D,Draw
609,1701,D2,2011,2011-08-15,Ein Frankfurt,Fortuna Dusseldorf,1,1,D,Draw
610,1702,D2,2011,2011-08-27,Aachen,Fortuna Dusseldorf,0,0,D,Draw


In [99]:
match_dates_unique=df['Date'].unique()

Selecting the unique dates of football matches to send weather requestes for. Want to minimise the requests

In [2]:
match_dates = np.sort(match_dates_unique)
#match_dates

NameError: name 'np' is not defined

In [102]:
def weather_dates(match_dates):
    for d in dates:
       # print("***********")
        print(d)

In [103]:
weather_dates(match_dates)

2012-03-31
2011-12-11
2011-08-13
2011-11-27
2012-02-18
2012-01-20
2012-02-04
2012-04-21
2011-09-18
2011-10-23
2011-10-01
2012-03-03
2011-08-27
2012-03-17
2011-11-06
2012-05-05
2012-04-11
2011-12-17
2012-02-03
2011-10-29
2012-01-22
2011-12-03
2012-04-14
2012-03-25
2012-03-10
2012-04-07
2011-11-19
2011-10-14
2011-09-24
2012-04-28
2011-12-18
2012-03-02
2012-03-16
2012-02-17
2011-08-06
2011-11-04
2011-09-16
2011-07-15
2012-05-06
2012-02-11
2011-10-02
2012-03-30
2011-10-22
2011-08-26
2011-08-07
2012-02-24
2011-07-17
2012-02-12
2011-08-22
2011-09-09
2012-04-01
2011-09-25
2012-04-20
2011-11-25
2012-03-09
2011-11-07
2011-09-10
2011-08-21
2011-09-26
2012-03-26
2011-10-30
2012-02-13
2012-02-26
2011-12-19
2012-03-11
2011-11-26
2012-04-23
2011-12-16
2011-11-05
2012-04-10
2012-04-15
2012-02-05
2012-03-04
2011-09-17
2012-01-21
2012-03-18
2012-04-22
2011-08-20
2012-02-10
2012-02-25
2011-12-10
2012-01-29
2011-10-16
2011-08-05
2012-03-23
2011-09-23
2012-01-28
2011-10-15
2012-03-24
2011-11-18
2012-04-29

### Dark Skies API

Data requirement from Dark Skies AP

Finally, let's get our client id and API key into our jupyter notebook.
If we remember that our file is just a regular JSON file, open the file and pull out the appropriate information from the ~/.secret/yelp_api.json file.

In [34]:
import json

#path = "~/.secret/dark_sky_api.txt"
path = 'c://Users/User1/.secret/dark_sky_api.txt'

def get_keys(path):
    with open(path) as f:
        return json.load(f)

In [43]:
keys = get_keys(path)
#print(keys)

In [36]:
api_key = keys['api_key']

In [44]:
#api_key